# Imports

In [33]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mutual_info_score
from sklearn.metrics import roc_auc_score, accuracy_score

# Load, display, first columns

In [34]:
leads = pd.read_csv("Leads.csv")
leads.head()

,Prospect ID,Lead Number,Lead Origin,Lead Source,Do Not Email,Do Not Call,Converted,TotalVisits,Total Time Spent on Website,Page Views Per Visit,...,Get updates on DM Content,Lead Profile,City,Asymmetrique Activity Index,Asymmetrique Profile Index,Asymmetrique Activity Score,Asymmetrique Profile Score,I agree to pay the amount through cheque,A free copy of Mastering The Interview,Last Notable Activity
0,7927b2df-8bba-4d29-b9a2-b6e0beafe620,660737,API,Olark Chat,No,No,0,0.0,0,0.0,...,No,Select,Select,02.Medium,02.Medium,15.0,15.0,No,No,Modified
1,2a272436-5132-4136-86fa-dcc88c88f482,660728,API,Organic Search,No,No,0,5.0,674,2.5,...,No,Select,Select,02.Medium,02.Medium,15.0,15.0,No,No,Email Opened
2,8cc8c611-a219-4f35-ad23-fdfd2656bd8a,660727,Landing Page Submission,Direct Traffic,No,No,1,2.0,1532,2.0,...,No,Potential Lead,Mumbai,02.Medium,01.High,14.0,20.0,No,Yes,Email Opened
3,0cc2df48-7cf4-4e39-9de9-19797f9b38cc,660719,Landing Page Submission,Direct Traffic,No,No,0,1.0,305,1.0,...,No,Select,Mumbai,02.Medium,01.High,13.0,17.0,No,No,Modified
4,3256f628-e534-4826-9d63-4a8b88782852,660681,Landing Page Submission,Google,No,No,1,2.0,1428,1.0,...,No,Select,Mumbai,02.Medium,01.High,15.0,18.0,No,No,Modified


In [35]:
leads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9240 entries, 0 to 9239
Data columns (total 37 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   Prospect ID                                    9240 non-null   object 
 1   Lead Number                                    9240 non-null   int64  
 2   Lead Origin                                    9240 non-null   object 
 3   Lead Source                                    9204 non-null   object 
 4   Do Not Email                                   9240 non-null   object 
 5   Do Not Call                                    9240 non-null   object 
 6   Converted                                      9240 non-null   int64  
 7   TotalVisits                                    9103 non-null   float64
 8   Total Time Spent on Website                    9240 non-null   int64  
 9   Page Views Per Visit                           9103 

# Data preparation and data cleaning

In [36]:
# Handle 'Select' values by replacing them with NaN
leads.replace('Select', np.nan, inplace=True)

# Drop columns with a high percentage of missing values (more than 30%)
missing_percentage = leads.isnull().sum() / len(leads) * 100
cols_to_drop = missing_percentage[missing_percentage > 30].index
leads.drop(cols_to_drop, axis=1, inplace=True)

# Impute missing values in numerical columns with the median
numerical_cols = leads.select_dtypes(include=np.number).columns
for col in numerical_cols:
    leads[col] = leads[col].fillna(leads[col].median())

# Impute missing values in categorical columns with the mode
categorical_cols = leads.select_dtypes(include=object).columns
for col in categorical_cols:
    leads[col] = leads[col].fillna(leads[col].mode()[0])

# Split the data into training, validation, and testing sets

In [37]:
leads_full_train, leads_test = train_test_split(leads, test_size=0.2, random_state=1)
leads_train, leads_val = train_test_split(leads_full_train, test_size=0.25, random_state=1)

In [38]:
# Reset indices
leads_train = leads_train.reset_index(drop=True)
leads_val = leads_val.reset_index(drop=True)
leads_test = leads_test.reset_index(drop=True)

In [39]:
# Separate the target variable
y_train = leads_train.Converted.values
y_val = leads_val.Converted.values
y_test = leads_test.Converted.values

In [40]:
# Remove the target
del leads_train['Converted']
del leads_val['Converted']
del leads_test['Converted']

# EDA

In [41]:
# Check the distribution of the target variable
global_conversion_rate = leads_full_train.Converted.mean()
print("Global Conversion Rate:", global_conversion_rate)

Global Conversion Rate: 0.38311688311688313


# Feature importance: Conversion rate and risk ratio, Mutual information

In [42]:
for c in categorical_cols:
    print(c)
    df_group = leads_full_train.groupby(c).Converted.agg(['mean', 'count'])
    df_group['diff'] = df_group['mean'] - global_conversion_rate
    df_group['risk'] = df_group['mean'] / global_conversion_rate
    display(df_group)
    print()
    print()

Prospect ID


,mean,count,diff,risk
Prospect ID,,,,
000104b9-23e4-4ddc-8caa-8629fe8ad7f4,1.0,1,0.616883,2.610169
0006d10a-eb01-4ba9-92e2-ad78588b2a40,0.0,1,-0.383117,0.000000
0011be30-fa97-465b-8e44-0ae83dff7eed,0.0,1,-0.383117,0.000000
0011f23e-9fd9-4256-b316-efc2e2639b0d,0.0,1,-0.383117,0.000000
001e6e14-2183-47ab-a405-108e44bc2e66,1.0,1,0.616883,2.610169
...,...,...,...,...
ffd99338-2e6b-4c3f-8650-68b94ea5e07f,0.0,1,-0.383117,0.000000
ffec8e24-0c99-4345-89f1-e3ad6689764f,1.0,1,0.616883,2.610169
fff076a3-fe95-4c79-9401-e15846be8086,0.0,1,-0.383117,0.000000




Lead Origin


,mean,count,diff,risk
Lead Origin,,,,
API,0.308414,2876,-0.074702,0.805014
Landing Page Submission,0.360677,3901,-0.022440,0.941427
Lead Add Form,0.919861,574,0.536744,2.400992
Lead Import,0.225000,40,-0.158117,0.587288
Quick Add Form,1.000000,1,0.616883,2.610169




Lead Source


...

,mean,count,diff,risk
I agree to pay the amount through cheque,,,,
No,0.383117,7392,0.0,1.0




A free copy of Mastering The Interview


,mean,count,diff,risk
A free copy of Mastering The Interview,,,,
No,0.39578,5071,0.012663,1.033053
Yes,0.35545,2321,-0.027667,0.927785




Last Notable Activity


,mean,count,diff,risk
Last Notable Activity,,,,
Approached upfront,1.000000,1,0.616883,2.610169
Email Bounced,0.160000,50,-0.223117,0.417627
Email Link Clicked,0.222222,135,-0.160895,0.580038
Email Marked Spam,1.000000,2,0.616883,2.610169
Email Opened,0.364362,2256,-0.018755,0.951046
Email Received,1.000000,1,0.616883,2.610169
Form Submitted on Website,0.000000,1,-0.383117,0.000000
Had a Phone Conversation,0.900000,10,0.516883,2.349153
Modified,0.230430,2721,-0.152687,0.601461


In [43]:
def mutual_info_conversion_score(series):
    return mutual_info_score(series, leads_full_train.Converted)

mi = leads_full_train[categorical_cols].apply(mutual_info_conversion_score)
mi.sort_values(ascending=False)

Prospect ID                                      0.665569
Last Activity                                    0.084629
Last Notable Activity                            0.072517
Lead Source                                      0.061448
Lead Origin                                      0.055599
What is your current occupation                  0.053127
Do Not Email                                     0.008911
Country                                          0.003266
A free copy of Mastering The Interview           0.000745
Through Recommendations                          0.000151
Digital Advertisement                            0.000131
Do Not Call                                      0.000130
Newspaper Article                                0.000130
Newspaper                                        0.000065
What matters most to you in choosing a course    0.000008
Search                                           0.000001
Magazine                                         0.000000
Receive More U

# Train the model

In [44]:
def train(X_train, y_train, C=1.0):
    # Convert DataFrame to a list of dictionaries
    train_dicts = X_train.to_dict(orient='records')  # Include all columns

    # Create a DictVectorizer object
    dv = DictVectorizer(sparse=False)

    # Fit the DictVectorizer object on the training data
    X_train_encoded = dv.fit_transform(train_dicts)

    # Create a LogisticRegression object
    model = LogisticRegression(C=C, max_iter=1000)

    # Fit the LogisticRegression object on the training data
    model.fit(X_train_encoded, y_train)

    return dv, model

In [45]:
dv, model = train(leads_full_train, leads_full_train.Converted.values, C=1.0)

# Evaluate the model

In [46]:
# Use the trained model to predict on the test set
y_pred = model.predict_proba(dv.transform(leads_test.to_dict(orient='records')))[:, 1]  # Include all columns

In [47]:
# Calculate the AUC score
auc = roc_auc_score(y_test, y_pred)
print(f"AUC Score: {auc:.4f}")

AUC Score: 0.8664
